## Demonstration of loading data, using no resolver

#### Add the path of modules of the project to sys

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

#### Import necessary libraries

In [4]:
import pathlib

from pepys_import.core.formats.repl_file import REPFile
from pepys_import.core.store.data_store import DataStore
from pepys_import.core.store.table_summary import TableSummary, TableSummarySet

#### Declaring global variables

In [3]:
RESOURCES_PATH = os.path.join('..', 'repl_files')

data_store = DataStore("", "", "", 0, ":memory:", db_type='sqlite')
data_store.initialise()
file_path = os.path.join(RESOURCES_PATH, "missing_platform.rep")
rep = REPFile(str(file_path))

______                      _                            _   
| ___ \                    (_)                          | |  
| |_/ /__ _ __  _   _ ___   _ _ __ ___  _ __   ___  _ __| |_ 
|  __/ _ \ '_ \| | | / __| | | '_ ` _ \| '_ \ / _ \| '__| __|
| | |  __/ |_) | |_| \__ \ | | | | | | | |_) | (_) | |  | |_ 
\_|  \___| .__/ \__, |___/ |_|_| |_| |_| .__/ \___/|_|   \__|
         | |     __/ | ______          | |                   
         |_|    |___/ |______|         |_|                   

Software Version :  0.0.4 


Database Type :  sqlite
Database Name :  :memory:
Database Host :  
-------------------------------------------------------------


In [6]:
with data_store.session_scope():
    datafile = data_store.get_datafile(rep.filepath, rep.datafile_type)
    for rep_line in rep.lines:
        platform = data_store.get_platform(rep_line.get_platform(),"Fisher", "UK", "Public")
        sensor = platform.get_sensor(data_store, "GPS")
        datafile.create_state(sensor, rep_line.timestamp)
    
    states_sum = TableSummary(data_store.session, data_store.db_classes.State)
    platforms_sum = TableSummary(
        data_store.session, data_store.db_classes.Platform
    )
    first_table_summary_set = TableSummarySet([states_sum, platforms_sum])
    print(first_table_summary_set.report("==Before=="))

AttributeError: 'NoneType' object has no attribute 'privacy_id'